In [1]:
import numpy as np
import pandas as pd
import shap
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, cohen_kappa_score
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood a

In [2]:
import tensorflow as tf
print(tf.__version__)

1.14.0


In [3]:
# Load the data
df1 = pd.read_csv("alldata36.csv", delimiter=',', header=0)


In [4]:
print(df1.head)


<bound method NDFrame.head of            COMNAM   SICCD  permno          public_date   gvkey        capei  \
0             NaN     NaN   10001  2014-09-30 00:00:00   12994    22.123579   
1             NaN     NaN   10001  2014-10-31 00:00:00   12994    21.261865   
2             NaN     NaN   10001  2014-11-30 00:00:00   12994    21.337626   
3             NaN     NaN   10001  2014-12-31 00:00:00   12994    20.447013   
4             NaN     NaN   10001  2015-01-31 00:00:00   12994    18.090596   
...           ...     ...     ...                  ...     ...          ...   
575887  TESLA INC  9999.0   93436  2021-08-31 00:00:00  184996 -1158.081248   
575888  TESLA INC  9999.0   93436  2021-09-30 00:00:00  184996 -1223.387579   
575889  TESLA INC  9999.0   93436  2021-10-31 00:00:00  184996 -1757.896419   
575890  TESLA INC  9999.0   93436  2021-11-30 00:00:00  184996 -2741.708578   
575891  TESLA INC  9999.0   93436  2021-12-31 00:00:00  184996 -2604.760993   

        peg_1yrforwar

In [5]:
# Convert 'public_date' to datetime and sort data
df1['public_date'] = pd.to_datetime(df1['public_date'])
df1 = df1.sort_values(by=['permno', 'public_date'])

# Get unique company counts before filtering
initial_bankrupt_count = df1[df1['Bankrupt']].drop_duplicates('permno').shape[0]
initial_non_bankrupt_count = df1[~df1['Bankrupt']].drop_duplicates('permno').shape[0]

# Specify the periods, adjusted for zero-based index
periods = [x - 1 for x in [3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36]]

# Filter rows for each 'permno' based on the specified periods
filtered_df1 = df1.groupby('permno').nth(periods).reset_index()

# Count unique bankrupt and non-bankrupt companies post-filtering
filtered_bankrupt_count = filtered_df1[filtered_df1['Bankrupt']].drop_duplicates('permno').shape[0]
filtered_non_bankrupt_count = filtered_df1[~filtered_df1['Bankrupt']].drop_duplicates('permno').shape[0]

# Print the initial and post-filtering counts
print(f"Initial Number of bankrupt companies: {initial_bankrupt_count}")
print(f"Initial Number of non-bankrupt companies: {initial_non_bankrupt_count}")
print(f"Filtered Number of bankrupt companies: {filtered_bankrupt_count}")
print(f"Filtered Number of non-bankrupt companies: {filtered_non_bankrupt_count}")

Initial Number of bankrupt companies: 2154
Initial Number of non-bankrupt companies: 13843
Filtered Number of bankrupt companies: 2154
Filtered Number of non-bankrupt companies: 13843


In [6]:
# Ratios
ratios = ['capital_ratio', 'equity_invcap', 'debt_invcap', 'totdebt_invcap', 'at_turn',
         'inv_turn', 'pay_turn', 'rect_turn', 'sale_equity', 'sale_invcap', 'sale_nwc',
         'invt_act', 'rect_act', 'fcf_ocf', 'ocf_lct', 'cash_debt', 'cash_lt', 'cfm',
         'short_debt', 'profit_lct', 'curr_debt', 'debt_ebitda', 'dltt_be', 'int_debt',
         'int_totdebt', 'lt_debt', 'lt_ppent', 'cash_conversion', 'cash_ratio', 'curr_ratio',
         'quick_ratio', 'accrual', 'rd_sale', 'adv_sale', 'staff_sale', 'efftax', 'gprof',
         'aftret_eq', 'aftret_equity', 'aftret_invcapx', 'gpm', 'npm', 'opmad', 'opmbd',
         'pretret_earnat', 'pretret_noa', 'ptpm', 'roa', 'roce', 'roe', 'de_ratio',
         'debt_assets', 'debt_at', 'debt_capital', 'intcov', 'intcov_ratio', 'dpr', 'bm',
         'capei', 'divyield', 'evm', 'pcf', 'pe_exi', 'pe_inc', 'pe_op_basic', 'pe_op_dil',
         'ps', 'ptb', 'peg_1yrforward', 'peg_ltgforward', 'peg_trailing']

In [7]:
# Prepare the dataset for LSTM
X, y = [], []
grouped = filtered_df1.groupby('permno')
sequence_length = 12

for _, group in grouped:
    group = group.sort_values(by='date')
    if len(group) >= sequence_length:
        X.append(group[ratios].tail(sequence_length).values)
        y.append(group['Bankruptcy'].iloc[-1])

X, y = np.array(X), np.array(y)

In [8]:
# Split the dataset based on the date ranges
train_mask = (filtered_df1['public_date'] >= '1970-01-01') & (filtered_df1['public_date'] <= '2010-12-31')
test_mask = (filtered_df1['public_date'] >= '1970-01-01') & (filtered_df1['public_date'] <= '2020-12-31')
out_sample_mask = (filtered_df1['public_date'] >= '2011-01-01') & (filtered_df1['public_date'] <= '2020-12-31')

In [9]:
X_train = np.array([X[i] for i in range(len(X)) if train_mask.iloc[i]])
y_train = np.array([y[i] for i in range(len(y)) if train_mask.iloc[i]])
X_test = np.array([X[i] for i in range(len(X)) if test_mask.iloc[i]])
y_test = np.array([y[i] for i in range(len(y)) if test_mask.iloc[i]])
X_out = np.array([X[i] for i in range(len(X)) if out_sample_mask.iloc[i]])
y_out = np.array([y[i] for i in range(len(y)) if out_sample_mask.iloc[i]])

# Count unique bankrupt and non-bankrupt companies in the training set
train_bankrupt_count = filtered_df1[train_mask & filtered_df1['Bankrupt']].drop_duplicates('permno').shape[0]
train_non_bankrupt_count = filtered_df1[train_mask & ~filtered_df1['Bankrupt']].drop_duplicates('permno').shape[0]

# Count unique bankrupt and non-bankrupt companies in the testing set
test_bankrupt_count = filtered_df1[test_mask & filtered_df1['Bankrupt']].drop_duplicates('permno').shape[0]
test_non_bankrupt_count = filtered_df1[test_mask & ~filtered_df1['Bankrupt']].drop_duplicates('permno').shape[0]

# Print the counts
print(f"Training data shape: {X_train.shape}, {y_train.shape}")
print(f"Testing data shape: {X_test.shape}, {y_test.shape}")
print(f"Training bankrupt companies: {train_bankrupt_count}")
print(f"Training non-bankrupt companies: {train_non_bankrupt_count}")
print(f"Testing bankrupt companies: {test_bankrupt_count}")
print(f"Testing non-bankrupt companies: {test_non_bankrupt_count}")

Training data shape: (12505, 12, 71), (12505,)
Testing data shape: (15261, 12, 71), (15261,)
Training bankrupt companies: 1983
Training non-bankrupt companies: 9460
Testing bankrupt companies: 2154
Testing non-bankrupt companies: 13843


In [10]:
# Impute missing values
imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test = imputer.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
X_out = imputer.transform(X_out.reshape(-1, X_out.shape[-1])).reshape(X_out.shape)

In [11]:
# Normalize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test_scaled = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
X_out_scaled = scaler.transform(X_out.reshape(-1, X_out.shape[-1])).reshape(X_out.shape)

In [12]:
# Balance the dataset using SMOTE
X_train_flattened = X_train_scaled.reshape(X_train_scaled.shape[0], -1)
sm = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = sm.fit_resample(X_train_flattened, y_train)
X_train_resampled = X_train_resampled.reshape(-1, sequence_length, X_train_scaled.shape[2])

In [13]:
# KFold and EarlyStopping setup
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [14]:
# Lists to store metrics
avg_accuracy = []
avg_recall = []
avg_precision = []
avg_f1 = []
avg_roc_auc = []
avg_kappa = []
avg_type_ii_error = []

In [15]:
# Start KFold training
fold_var = 1
for train_index, val_index in kf.split(X_train_resampled):
    X_train_fold, X_val_fold = X_train_resampled[train_index], X_train_resampled[val_index]
    y_train_fold, y_val_fold = y_train_resampled[train_index], y_train_resampled[val_index]
    
    # Build the LSTM model
    model = Sequential([
        LSTM(50, input_shape=(sequence_length, len(ratios)), activation='tanh'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train_fold, y_train_fold, validation_data=(X_val_fold, y_val_fold),
                        epochs=20, batch_size=32, callbacks=[early_stopping])

    # Predictions for this fold
    y_pred = model.predict(X_val_fold)
    y_pred_binary = (y_pred > 0.5).astype(int)

    # Calculate metrics for this fold
    accuracy = accuracy_score(y_val_fold, y_pred_binary)
    recall = recall_score(y_val_fold, y_pred_binary)
    precision = precision_score(y_val_fold, y_pred_binary)
    f1 = f1_score(y_val_fold, y_pred_binary)
    roc_auc = roc_auc_score(y_val_fold, y_pred)
    kappa = cohen_kappa_score(y_val_fold, y_pred_binary)
    type_ii_error = 1 - recall

    print(f'Fold {fold_var}:')
    print(f'  Accuracy: {accuracy:.4f}')
    print(f'  Recall: {recall:.4f}')
    print(f'  Precision: {precision:.4f}')
    print(f'  F1 Score: {f1:.4f}')
    print(f'  ROC AUC: {roc_auc:.4f}')
    print(f'  Kappa: {kappa:.4f}')
    print(f'  Type II Error: {type_ii_error:.4f}')
    print()

    # Append metrics for this fold to the lists
    avg_accuracy.append(accuracy)
    avg_recall.append(recall)
    avg_precision.append(precision)
    avg_f1.append(f1)
    avg_roc_auc.append(roc_auc)
    avg_kappa.append(kappa)
    avg_type_ii_error.append(type_ii_error)

    fold_var += 1


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 17355 samples, validate on 4339 samples
Epoch 1/20
17355/17355 [==============================] - 18s 1ms/sample - loss: 0.3902 - acc: 0.8328 - val_loss: 0.3383 - val_acc: 0.8594
Epoch 2/20
17355/17355 [==============================] - 17s 995us/sample - loss: 0.2886 - acc: 0.8836 - val_loss: 0.2778 - val_acc: 0.8878
Epoch 3/20
17355/17355 [==============================] - 17s 990us/sample - loss: 0.2345 - acc: 0.9121 - val_loss: 0.2456 - val_acc: 0.9115
Epoch 4/20
17355/17355 [==============================] - 18s 1ms/sample - loss: 0.1951 - acc: 0.9288 - val_loss: 0.2144 - val_acc: 0.9221
Epoch 5/20
17355/17355 [==============================] - 17s 1ms/sample - loss: 0.1586 - acc: 0.9435 - val_loss: 0.2220 - val_acc: 0.9221
Epoch 6/20
17355/17355 [================

In [16]:
# Calculate average of all folds
print('Average metrics across all folds:')
print(f'  Average Accuracy: {np.mean(avg_accuracy):.4f}')
print(f'  Average Recall: {np.mean(avg_recall):.4f}')
print(f'  Average Precision: {np.mean(avg_precision):.4f}')
print(f'  Average F1 Score: {np.mean(avg_f1):.4f}')
print(f'  Average ROC AUC: {np.mean(avg_roc_auc):.4f}')
print(f'  Average Kappa: {np.mean(avg_kappa):.4f}')
print(f'  Average Type II Error: {np.mean(avg_type_ii_error):.4f}')

Average metrics across all folds:
  Average Accuracy: 0.9258
  Average Recall: 0.9488
  Average Precision: 0.9071
  Average F1 Score: 0.9274
  Average ROC AUC: 0.9683
  Average Kappa: 0.8515
  Average Type II Error: 0.0512


In [17]:
model.save('LSTM_Model_72_Q36.h5')

In [13]:
# Load the model
model = load_model('LSTM_Model_72_Q36.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [14]:
# Predict on the train set
y_train_pred_probs = model.predict(X_train_scaled)
y_train_pred = (y_train_pred_probs > 0.5).astype(int)


In [15]:
# Calculate metrics
train_accuracy = accuracy_score(y_train, y_train_pred)
train_precision = precision_score(y_train, y_train_pred)
train_recall = recall_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred)
train_roc_auc = roc_auc_score(y_train, y_train_pred_probs)

# Print the metrics
print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Train Precision: {train_precision:.4f}")
print(f"Train Recall: {train_recall:.4f}")
print(f"Train F1 Score: {train_f1:.4f}")
print(f"Train ROC AUC: {train_roc_auc:.4f}")


Train Accuracy: 0.9370
Train Precision: 0.6880
Train Recall: 0.9602
Train F1 Score: 0.8016
Train ROC AUC: 0.9802


In [16]:
# Compute and print the confusion matrix
cm = confusion_matrix(y_train, y_train_pred)
print("Confusion Matrix:")
print(cm)


Confusion Matrix:
[[10125   722]
 [   66  1592]]


In [17]:
# Predict on the test set
y_test_pred_probs = model.predict(X_test_scaled)
y_test_pred = (y_test_pred_probs > 0.5).astype(int)

In [18]:
# Calculate metrics
test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred)
test_recall = recall_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)
test_roc_auc = roc_auc_score(y_test, y_test_pred_probs)

# Print the metrics
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test F1 Score: {test_f1:.4f}")
print(f"Test ROC AUC: {test_roc_auc:.4f}")


Test Accuracy: 0.9247
Test Precision: 0.6578
Test Recall: 0.9140
Test F1 Score: 0.7650
Test ROC AUC: 0.9685


In [19]:
# Compute and print the confusion matrix
cm = confusion_matrix(y_test, y_test_pred)
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[12242   973]
 [  176  1870]]


In [20]:
# Predict on the Out of Sample set
y_out_pred_probs = model.predict(X_out_scaled)
y_out_pred = (y_out_pred_probs > 0.5).astype(int)

In [22]:
# Calculate metrics
Out_accuracy = accuracy_score(y_out, y_out_pred)
Out_precision = precision_score(y_out, y_out_pred)
Out_recall = recall_score(y_out, y_out_pred)
Out_f1 = f1_score(y_out, y_out_pred)
Out_roc_auc = roc_auc_score(y_out, y_out_pred_probs)

# Print the metrics
print(f"Out of Sample Accuracy: {Out_accuracy:.4f}")
print(f"Out of Sample Precision: {Out_precision:.4f}")
print(f"Out of Sample Recall: {Out_recall:.4f}")
print(f"Out of Sample F1 Score: {Out_f1:.4f}")
print(f"Out of Sample ROC AUC: {Out_roc_auc:.4f}")


Out of Sample Accuracy: 0.8690
Out of Sample Precision: 0.5255
Out of Sample Recall: 0.7165
Out of Sample F1 Score: 0.6063
Out of Sample ROC AUC: 0.9116


In [23]:
# Compute and print the confusion matrix
cm = confusion_matrix(y_out, y_out_pred)
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[2117  251]
 [ 110  278]]
